# Laboratory notebook
This notebook is where you can experiment using the subtractive solver on multiple repositories yourself!
Add the repo url (from either a remote git repository, or a local path) and the branch you want to analyze (defaults to main).
I have set up an example setup that allows you to just add repo paths to a list and the notebook will concurrently analyze all the 
repositories and display them in a chart.

In [1]:
from dataclasses import dataclass
from main import main

@dataclass
class ConfigurationItem():
    repo_path: str
    branch: str = "main"